In [1]:
import os

import SimpleITK as sitk
import six
from pathlib import Path
import pandas as pd
import json
import numpy as np
import re
from tqdm.notebook import tqdm

from radiomics import featureextractor, getTestCase

In [2]:
radiomic_modality = "t1"
drive_root = Path("/mnt/h")

In [3]:
inference_root = drive_root / "srs-9" / "3Tpioneer_bids_predictions"
dataroot = drive_root / "3Tpioneer_bids"
image_file = f"{radiomic_modality}.nii.gz"

In [4]:
data_file = "/home/srs-9/Projects/ms_mri/analysis/paper1/data0/t1_data_full.csv"
df = pd.read_csv(data_file)
df = df.set_index("subid")

In [5]:
image_labels = []
for subid, row in df.iterrows():
    image_path = dataroot / str(row['scan_folder']) / image_file
    if re.match("choroid_t1_flair", str(row['label'])):
        label_path = dataroot / str(row['scan_folder']) / str(row['label'])
    else:
        label_path = inference_root / str(row['scan_folder']) / str(row['label'])
    try:
        assert image_path.exists() and label_path.exists()
    except AssertionError:
        print(subid)
        print(str(image_path), image_path.exists())
        print(str(label_path), label_path.exists())
        continue
    image_labels.append((subid, str(image_path), str(label_path)))

1196
/mnt/h/3Tpioneer_bids/nan/t1.nii.gz False
/mnt/h/srs-9/3Tpioneer_bids_predictions/nan/nan False


In [7]:
for i in tqdm(range(4)):
    pass

  0%|          | 0/4 [00:00<?, ?it/s]

In [9]:
label_nums = {'choroid': 1, 'pineal': 2, 'pituitary': 3}
extractor = featureextractor.RadiomicsFeatureExtractor()

struct = "pituitary"
subjects = []
results = []
failed_subs = []
for subid, image, seg in tqdm(image_labels, total=len(image_labels)):
    try:
        results.append(extractor.execute(image, seg, label=label_nums[struct]))
    except Exception as e:
        failed_subs.append((subid, e))
        continue
    else:
        subjects.append(subid)

  0%|          | 0/574 [00:00<?, ?it/s]

GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Average, only 1 needs to be calculated
GLCM is symmetrical, therefore Sum Average = 2 * Joint Avera

In [12]:
def result_arrays2list(result):
    for k, v in result.items():
        if isinstance(v, np.ndarray):
            result[k] = v.tolist()
    return result

In [10]:
results_backup = results.copy()

In [ ]:
results = [result_arrays2list(result) for result in results]

radiomics_df = pd.DataFrame(results, index=subjects)
radiomics_df.to_json(f"{struct}-{radiomic_modality}-radiomics_full.json", indent=4)

In [14]:
radiomics_df.to_json(f"{struct}-{radiomic_modality}-radiomics.json")

In [16]:
columns = radiomics_df.columns
keep_cols = [col for col in columns if re.match("original", col)]

In [21]:
rad_df = radiomics_df[keep_cols]
rad_df.index.name = "subid"
rad_df.to_json(f"{struct}-{radiomic_modality}-radiomics.json")
rad_df.to_csv(f"{struct}-{radiomic_modality}-radiomics.csv")